**Authentic News Platform Generator**

Our code runs in 3 sets:
1. Trend Extraction using Tweepy
2. News Extraction using the trend as keyword
3. News Text Authentication using Machine Learning

1. Trend Extraction using tweepy
Tweepy is an open source Python package that gives you a very convenient way to access the Twitter API with Python. Tweepy includes a set of classes and methods that represent Twitter's models and API endpoints, and it transparently handles various implementation details, such as: Data encoding and decoding.

Please note: In order to use tweepy, you just have Elevated Twitter Developer Access. For this, there is an elaborate application process which requires upto 2 weeks to get approved. 
Once you get access, you can use your bearer tokens for freely accessing your concerned dataset which in our case is Top 50 Trends. 

In [ ]:
!pip install pygooglenews
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 128 kB 49.3 MB/s 
     |████████████████████████████████| 1.2 MB 41.0 MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44951 sha256=d8acccc781c7c26265cfeb7fb069489085b23102820477afc53f2020b72faf8a
  Stored in directory: /root/.cache/pip/wheels/29/bf/46/b4a597d435d3aee6c2fa583824897336d65abf13ebe3405b70
Successfully built feedparser
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Looking in indexes: https://py

In [ ]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 211 kB 4.0 MB/s 
     |████████████████████████████████| 93 kB 1.8 MB/s 
     |████████████████████████████████| 7.4 MB 14.2 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=e6a8b3d58d645a980b9c221e1c9aaaceeaee97dc0d26d70448a799c12854ef1e
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=83028c2a21a87dfb180e941d9e5fd41a02e27309d33e757dd2b35fde7c246164
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398405 sha256=46d5c669810fe16ca3f77e4e8b3f1702fb664d181a2f726371b28f248468901f
  Stored in directory: /root/.cache/pip/

In [ ]:
import tweepy

#Tokens Generated for Twitter API
apikey = '0udfbvMcxXr7JBQdOIvvG3CVU'
apisecretkey = 'Bax2Uo5vXPnXG8wPOeGwP0Q8vh3cZSRiBYj61t7iUS6HBDTkDj'
accesskey = '1586524368007532544-7FK18bZ0lYZQcVxjvLkT0FrzhhPsJL'
accesssecret = 'BFsE945AE1gWkGhHdajZp8iYI4T7IjjyhlJA7NRuK20dD'
auth = tweepy.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesskey,accesssecret)
api = tweepy.API(auth)

# WOEID = Where On Earth ID; used to identify the location
WOEID = 2358820

top_trends = api.trends_place(WOEID)
top_trends = top_trends[0].get("trends")

#Top 50 trends are being extracted
print(len(top_trends))
#Trend #1
print(top_trends[0])

lookup = []
for i in top_trends:
  lookup.append(i.get("name"))

print(lookup)

50
{'name': '#GivingTuesday', 'url': 'http://twitter.com/search?q=%23GivingTuesday', 'promoted_content': None, 'query': '%23GivingTuesday', 'tweet_volume': 184625}
['#GivingTuesday', 'Klay', 'Louisville', 'Mario', '#QueenSugar', 'Nintendo', '#WWENXT', 'Peach', 'Panda', 'Luigi', 'Oath Keepers', 'Randle', '#USMNT', 'Bellarmine', 'Respect for Marriage Act', 'Wiggins', 'Iran', '#MakeAMovieChristmas', 'Kuminga', 'Stewart Rhodes', 'Bowser', 'Netherlands', 'Taylor Hall', 'Pulisic', 'McConnell', 'Smash', 'Casey Anthony', 'Donkey Kong', 'Dutch', 'Wheeler', 'Jack Black', 'Marquette', 'Anthony Lamb', 'Charlie Day', 'Boo Corrigan', 'Darla', 'Ralph Angel', 'Obergefell', 'Seditious Conspiracy', 'VGBC', 'Dijak', 'Liver King', 'Tua vs Herbert', 'The Senate', 'Rainbow Road', 'Calipari', 'Joe Girard', 'Judah Mintz', 'Shelby Miller', '12 Republicans']


**2. News Extraction using the trend as keyword**
For this purpose we are using pygooglenews and beautifulsoup libraries.

a. pygooglenews: A Python wrapper of the Google News RSS feed. It can be used for extracting top stories, topic-related news feeds, geolocation news feed, and an extensive full-text search feed.

b. Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

To get the access to all the functions, we first have to initiate the GoogleNews class.It has 2 required variables: lang and country for which we choose English (en) and United States (US) respectively.

Then we initialize an empty dictionary which stores the url of the 5 most relevant articles associated with the trend keyword. For this purpose we use gn.search function. Google News search itself is a complex function that has inherited some features from the standard Google Search. The returned object contains feed and entries list of articles found with all data parsed.

In [ ]:
from pygooglenews import GoogleNews
gn = GoogleNews(lang = 'en', country = 'US')

newslinks = {}
for i in lookup:
  s = gn.search(i)
  links = set()
  j = 0
  for entry in s["entries"]:
    if j <= 4:
      links.add(entry["link"])
    j += 1
  newslinks[i]=links

print(newslinks)

{'#GivingTuesday': {'https://news.google.com/__i/rss/rd/articles/CBMiaGh0dHBzOi8vcmljaG1vbmRzdGFuZGFyZC5jb20vcmljaG1vbmQvMjAyMi8xMS8yOS9jaGV2cm9uLWRvbmF0ZXMtMTAway10by0xNS1sb2NhbC1vcmdzLW9uLWdpdmluZ3R1ZXNkYXkv0gFsaHR0cHM6Ly9yaWNobW9uZHN0YW5kYXJkLmNvbS9yaWNobW9uZC8yMDIyLzExLzI5L2NoZXZyb24tZG9uYXRlcy0xMDBrLXRvLTE1LWxvY2FsLW9yZ3Mtb24tZ2l2aW5ndHVlc2RheS9hbXAv?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiN2h0dHBzOi8vd3d3LmRpc3RyYWN0aWZ5LmNvbS9wL2hpc3Rvcnktb2YtZ2l2aW5nLXR1ZXNkYXnSAQA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMifmh0dHBzOi8vd2hhdi5uZXQvMjAyMi8xMS8yOS9naXZlLWEtZ2lmdC1pbi1zdXBwb3J0LW9mLWxvY2FsLW5ld3MtYW5kLXByb2dyYW1zLW9uLWdpdmluZ3R1ZXNkYXktZWFybHktY29udHJhY3RvcnMtb2ZmZXJzLW1hdGNoL9IBAA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiWGh0dHBzOi8vY2l2aWxlYXRzLmNvbS8yMDIyLzExLzI5LzIyLXJlYXNvbnMtdG8tc3VwcG9ydC1jaXZpbC1lYXRzLW9uLWdpdmluZ3R1ZXNkYXktMjAyMi_SAQA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiVGh0dHBzOi8vd3d3LmdyZWVubWF0dGVycy5jb20

data.csv file is going to be our main input file. It contains all the information which we derived from the news articles which are related to the top trends. The most important field for us here is the text field since that helps us to define the authenticity of the article.

In [ ]:
import newspaper
import json
import csv

with open('data.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
  writer.writerow(['tag','title','date','link','text','label'])
  for key in newslinks:
    for entry in newslinks[key]:
      url = entry
  
      try:
        article = newspaper.Article(url=url, language='en')
        article.download()
        article.parse()
        
        article ={
            "tag": key,
            "title": str(article.title),
            "text": str(article.text),
            "link": url,
            "date": str(article.publish_date),
        }
        if article["text"] != '':
          writer.writerow([article['tag'],article['title'],article['date'],article['link'],article['text']])
      except:
        print('unusable article found, aborting parse attempt and continuing to next article')


unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to next article
unusable article found, aborting parse attempt and continuing to

**3. News Text Authentication using Machine Learning**
After we are done with transforming and cleaning the data for the ML code, we start by training the algorithm. We do this by using 2 authenticated flat files available online. 

To download the 2 flat files, we download the zip file using the below given code:

In [ ]:
!gdown --id 1nynHsLl_i-AtKGgChKHVrMGETOmlT4VU
!gdown --id 1o4sLlG42jUBevMb7g_VFNf9R-OYdExD_


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1nynHsLl_i-AtKGgChKHVrMGETOmlT4VU
To: /content/True.csv
100% 53.6M/53.6M [00:00<00:00, 197MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1o4sLlG42jUBevMb7g_VFNf9R-OYdExD_
To: /content/Fake.csv
100% 62.8M/62.8M [00:00<00:00, 68.8MB/s]


For data manipulation and dataframe formulation, we use the pandas library. We are then going to consolidate a portion of both files inside of a frame along with a boolean label. So ultimately we end up with a dataset of 10,000 records which is labeled to predict the record's accuracy.

In [ ]:
import pandas as pd

true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')
true.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [ ]:
true['label'] = 1
fake['label'] = 0

# Combine the sub-datasets in one.
frames = [true.loc[:5000][:], fake.loc[:5000][:]]
df = pd.concat(frames)
df.tail()

,title,text,subject,date,label
4996,Justice Department Announces It Will No Longe...,Republicans are about to lose a huge source of...,News,"August 18, 2016",0
4997,WATCH: S.E. Cupp Destroys Trump Adviser’s ‘Fa...,A pawn working for Donald Trump claimed that w...,News,"August 18, 2016",0
4998,WATCH: Fox Hosts Claim Hillary Has Brain Dama...,Fox News is desperate to sabotage Hillary Clin...,News,"August 18, 2016",0
4999,CNN Panelist LAUGHS In Corey Lewandowski’s Fa...,As Donald Trump s campaign continues to sink d...,News,"August 18, 2016",0
5000,Trump Supporter Who Wants To Shoot Black Kids...,"Hi folks, John Harper here, at least if you as...",News,"August 18, 2016",0


The PassiveAggresive Classifier does not accept null values as input, so we clean the records we feed to the model by removing it and reordering the dataframe for clarity.

In [ ]:
X = df.drop('label', axis=1) 
y = df['label']
# Delete missing data
df = df.dropna()
df2 = df.copy()
df2.reset_index(inplace=True)

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
nltk.download('stopwords')
ps = PorterStemmer()
corpus = []
for i in range(0, len(df2)):
    review = re.sub('[^a-zA-Z]', ' ', df2['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_v.fit_transform(corpus).toarray()
y = df2['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

We test out the accuracy of the trained model in hand from a test value derived from the flat files which we had initially. Since we got an accuracy of 99%, we can then move forward and use it for getting a valid result.

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
import numpy as np
import itertools
classifier = PassiveAggressiveClassifier(max_iter=1000)
classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.999


After loading the model and vectorizor, the code assigns a 0 or 1 to the left of a linked article, allowing the user to scroll through the list in the console to select an article of their choosing.

In [ ]:
# Tokenization
data1 = pd.read_csv('data.csv')
x = 0
new =[]

for x in range(len(data1['text'])):
  review = re.sub('[^a-zA-Z]', ' ', data1['text'][x])
  review = review.lower()
  review = review.split() 
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  # Vectorization
  val = tfidf_v.transform([review]).toarray()
  # Predict 
  a = classifier.predict(val)
  new.append(a[0])
  print(a[0],data1['link'][x])

  print(a[0])


FileNotFoundError: ignored